In [2258]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math

In [2259]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [2260]:
def prepare_data(file_name='./train.csv'):
    train_df = pd.read_csv(file_name)
    train_df.drop(train_df.columns[0],axis=1,inplace=True)
    train_npa = train_df.to_numpy()
    
    train_data = []
    max_item_id = 0
    max_user_id = train_npa.shape[0]

    for i, docs in enumerate(train_npa):
        tmp_arr = np.array(list(map(int, list(docs)[0].split())))
        tmp_max = np.max(tmp_arr)
        if tmp_max > max_item_id:
            max_item_id = tmp_max
        train_data.append(tmp_arr)

    train_data = np.array(train_data, dtype=object)

    return train_data, max_item_id, max_user_id

In [2261]:
def create_rating_matrix(train_data, max_item_id, max_user_id):
    R = np.zeros((max_item_id, max_user_id))
    
    for i in range(max_item_id):
        for j in range(max_user_id):
            if i in train_data[j]:
                # user j interacted with item i
                R[i][j] = 1
    
    return R

In [2262]:
def bce_loss(data, i, j):
    hor_sum = np.sum(data, axis=1)
    vert_sum = np.sum(data, axis=0)
    
    return -(data[i, j]*np.ln(hor_sum[i]) + (1-data[i, j])*np.ln(1-vert_sum[j]))

In [2263]:
# def gradient_descent(start, learn_rate=0.01, n_iter=50, tolerance=1e-06):
#     vector = start
#     for _ in range(n_iter):
# #         diff = -learn_rate * np.gradient(vector)
#         if np.all(np.abs(diff) <= tolerance):
#             break
#         vector += diff
#     return vector

In [2264]:
def count_occurrences(row):
    return len([x for x in row if x == 1])

In [2265]:
# def gradient_descent(R, P, Q, max_iter=1, learn_rate=0.01, n_iter=50, tolerance=1e-06, lamb1=0.001, lamb2=0.001):
#     for iters in range(max_iter):
#         print("iteration: {}".format(iters))
#         for x in range(R.shape[1]):
#             for i in range(R.shape[0]):
# #                 o = count_occurrences()
# #                 eps = R[i, x] - (np.dot(Q[i], P[x]))
# #                 eps = R[i, x] - (np.log(sigmoid(Q[i])) + np.log(sigmoid(P[x])))
#                 p = sigmoid(np.dot(Q[i], P[x]))
# #                 print(p)
#                 if math.isnan(p):
#                     print("({}, {}) --- Q[i]: {}, P[x]: {}".format(x, i, Q[i], P[x]))
#                 if p == 0:
#                     p = 0.01
#                 if (1-p) <= 0:
#                     p = 0.99
#                 y = R[i, x]
# #                 print(p)
                
#                 try:
#                     eps = -(np.sum(y*np.log(p) + (1-y)*(np.log(1-p))))
# #                     print(eps)
#                     if math.isinf(eps) or math.isnan(eps):
#                         print("eps is wrong --- y: {}, p: {}".format(y, p))
#                 except:
#                     print("p: {}".format(p))
#                     print("y: {}".format(y))
# #                 eps = -(R[i, x]*np.log(pred) + (1-R[i, x])*np.log((1-pred)))

#                 if (y-p) <= 0:
#                     eps = -eps
#                 Q[i] = Q[i] + learn_rate*(eps*P[x]-lamb2*Q[i])
# #                 print(learn_rate*(eps*Q[i]-lamb1*P[x]))
#                 P[x] = P[x] + learn_rate*(eps*Q[i]-lamb1*P[x])
    
#     return P, Q

In [2266]:
def gradient_descent(R, P, Q, max_iter=50, learn_rate=0.01, lamb1=0.001, lamb2=0.001):
    for iters in range(max_iter):
        print("iteration: {}".format(iters))
        
        r = np.random.randint(0, Q.shape[0])
        for i in range(P.shape[0]):
            y = R[r, i]
            p = np.dot(Q[r], P[i])
            eps = (-y/p)+((1-y)/(1-p))
            eps = np.sign(p)*eps

            Q[r] = Q[r] + learn_rate*(-eps*P[i]-lamb2*Q[r])
            P[i] = P[i] + learn_rate*(-eps*Q[r]-lamb1*P[i])
            
            
            
        r = np.random.randint(0, P.shape[0])
        for i in range(Q.shape[0]):
#             eps = R[i, r] - (np.dot(Q[i], P[r]))
            y = R[i, r]
            p = np.dot(Q[i], P[r])
            eps = (-y/p)+((1-y)/(1-p))
            
            eps = np.sign(p)*eps
    
            Q[i] = Q[i] + learn_rate*(-eps*P[r]-lamb2*Q[i])
            P[r] = P[r] + learn_rate*(-eps*Q[i]-lamb1*P[r])
            
#             print("before: y: {} p: {}, after: {}".format(y, p, np.dot(Q[i], P[r])))
    
    return P, Q

In [2267]:
# def gradient_descent(R, P, Q, max_iter=1, learn_rate=0.1, tolerance=1e-06, lamb1=0.01, lamb2=0.01):
#     for iters in range(max_iter):
#         for x in range(R.shape[1]):
#             for i in range(R.shape[0]):
        
#                 y = R[i, x]
#                 p = np.dot(Q[i], P[x])
# #                 print("y: {}, p: {}".format(y, p))
# #                 if p > 1:
# #                     print(p)
# #                 if p >= 1:
# #                     p = 0.99
# #                 if p < 0:
# #                     p = 0.99

#                 eps = (-y/p)+((1-y)/(1-p))
# #                 eps = np.sign(eps)
# #                 print(eps)
#                 eps = np.sign(p)*eps
#                 Q[i] = Q[i] + learn_rate*(-eps*P[x]-lamb2*Q[i])
#                 P[x] = P[x] + learn_rate*(-eps*Q[i]-lamb1*P[x])

# #                 print("before: y: {} p: {}, after: {}".format(y, p, np.dot(Q[i], P[x])))
                
#     return P, Q

In [2268]:
def get_most_relevant(mat, n=50):
    res = []
    
    for i in range(mat.shape[0]):
        d = dict(enumerate(mat[i]))
        sl = sorted(d.items(), key=lambda item: item[1], reverse=True)
        res.append(sl[:n])

    return res

In [2269]:
def output_result_csv(rel_items):        
#     res = pd.DataFrame.from_dict(rel_items, orient='index', columns=['ItemId'])
    for k, v in rel_items.items():
        rel_items[k] = ' '.join(str(x) for x in v)
        
    res = pd.DataFrame([{'ItemId': v} for v in rel_items.values()], index=rel_items.keys())
    res.index.name = 'UserId'
    
    res.to_csv('./submission.csv')  
        

In [2270]:
def bpr_sgd(R, max_iter=1000, learn_rate=0.001, lamb=0.001):
    Q = np.random.uniform(0, 1, size=(R.shape[1], 2))
    P = np.random.uniform(0, 1, size=(R.shape[0], 2))
    
    for it in range(max_iter):
#         print("iter: {}".format(it))
        u = np.random.randint(0, R.shape[1])
        i = np.random.randint(0, R.shape[0])
        j = np.random.randint(0, R.shape[0])
        # < 0: prefers j
        # > 0: prefers i
#         xuij = np.dot(Q[u], P[i]) - np.dot(Q[u], P[j])
        xuij = np.dot()
    
        # Q (W)
        d = (P[i]-P[j])
        Q[u] = Q[u] + learn_rate*(((np.exp(-xuij))/(1+np.exp(-xuij))) * d + lamb*Q[u])
        
        # Pi (H)
        d = Q[u]
        P[i] = P[i] + learn_rate*(((np.exp(-xuij))/(1+np.exp(-xuij))) * d + lamb*P[i])
        
        # Pj (H)
        d = -Q[u]
        P[j] = P[j] + learn_rate*(((np.exp(-xuij))/(1+np.exp(-xuij))) * d + lamb*P[j])
    
    return P, Q

In [2271]:
train_data, max_item_id, max_user_id = prepare_data()

In [2272]:
# R = create_rating_matrix(train_data, max_item_id, max_user_id)

In [2273]:
M, N = R.shape[0], R.shape[1]

In [2274]:
# u, s, vh = np.linalg.svd(R, )

# # P = u
# # Q = np.matmul(s, vh)

In [2275]:
# P = np.matmul(u, s)
# Q = vh

In [2276]:
# initial_Q = np.random.uniform(0, 1, size=(M, 2))
# initial_P = np.random.uniform(0, 1, size=(N, 2))

In [2277]:
# initial_Q = np.random.uniform(0, 1, size=(M, 2))
# initial_P = np.random.uniform(0, 1, size=(N, 2))
# P, Q = gradient_descent(R, initial_P, initial_Q)

In [2278]:
P, Q = bpr_sgd(R)

In [2279]:
M = np.matmul(P, np.transpose(Q))

In [2280]:
relevant_items = get_most_relevant(M)
rel_items = [[x[0] for x in l] for l in relevant_items]

In [2283]:
rel_items

[[2188,
  3148,
  373,
  2974,
  3348,
  3096,
  1601,
  3899,
  2055,
  1438,
  3653,
  2189,
  4207,
  2728,
  3451,
  4315,
  3283,
  1542,
  124,
  2530,
  2696,
  4282,
  1852,
  736,
  2956,
  4378,
  491,
  1322,
  1537,
  772,
  2391,
  2203,
  114,
  1507,
  4153,
  2223,
  3132,
  2827,
  3039,
  141,
  2720,
  970,
  2057,
  3569,
  852,
  1921,
  241,
  3274,
  2923,
  1862],
 [1601,
  2188,
  2974,
  3148,
  4282,
  4207,
  2189,
  3283,
  373,
  3653,
  3096,
  852,
  1507,
  114,
  124,
  3675,
  18,
  3348,
  736,
  4112,
  3396,
  3614,
  1568,
  3899,
  3572,
  2720,
  2522,
  3451,
  2822,
  1542,
  2391,
  2956,
  3303,
  2991,
  2728,
  4315,
  2055,
  3058,
  470,
  970,
  241,
  1438,
  2203,
  3684,
  1760,
  3554,
  449,
  102,
  4405,
  2953],
 [373,
  3348,
  2055,
  1438,
  3148,
  2188,
  3899,
  3274,
  1322,
  2696,
  3096,
  2974,
  491,
  2837,
  3912,
  1862,
  41,
  2728,
  3046,
  3516,
  22,
  2530,
  2057,
  496,
  3225,
  3288,
  1537,
  3653,
  4

In [2281]:
rel_items_dict = dict(enumerate(rel_items))

In [2282]:
output_result_csv(rel_items_dict)